In [ ]:
#CELL 1
#@title Keep this widget playing to prevent Colab from disconnecting you { display-mode: "form" }
#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
import requests
import tarfile
import os
from tqdm import tqdm
import threading
import time

#@title Koboldcpp Colab (Improved Edition)

# URL of the built koboldcpp folder
url = "https://huggingface.co/kalomaze/ColabDependencies/resolve/main/content/koboldcpp.tar.gz"

Model = "MythoMax-L2-13B-GGUF" #@param ["MythoMax-L2-13B-GGUF", "ReMM-SLERP-L2-13B-GGUF", "Stheno-L2-13B-GGUF"]
Quant_Method = "5_K_M" #@param ["4_K_M", "5_K_M"]
Layers = 43 #@param [43]{allow-input: true}
Context = 4096 #@param [4096]{allow-input: true}

#@markdown ### OPTIONAL: Manual Model Link
Use_Manual_Model = True #@param {type:"boolean"}
Manual_Link = "" #@param {type:"string"}

#@markdown ### OPTIONAL: Build Latest Kobold (takes ~7 minutes)
Force_Update_Build = False #@param {type:"boolean"}

model_links = {
    "MythoMax-L2-13B-GGUF": "https://huggingface.co/TheBloke/MythoMax-L2-13B-GGUF/resolve/main/mythomax-l2-13b.Q{}.gguf",
    "ReMM-SLERP-L2-13B-GGUF": "https://huggingface.co/TheBloke/ReMM-SLERP-L2-13B-GGUF/resolve/main/remm-slerp-l2-13b.Q{}.gguf",
    "Stheno-L2-13B-GGUF": "https://huggingface.co/TheBloke/Stheno-L2-13B-GGUF/resolve/main/stheno-l2-13b.Q{}.gguf"
}

if Use_Manual_Model:
    if Manual_Link.strip() != "":
        print(f"\nManual Model detected; will use {Manual_Link} instead of {Model}\n")
        Model = Manual_Link
    else:
        print(f"\nWarning: Manual Model enabled, but no link was found. Falling back to {Model}\n")
        if Model in model_links:
            Model = model_links[Model].format(Quant_Method)
else:
    if Model in model_links:
        Model = model_links[Model].format(Quant_Method)

# Ensure the directory exists
if not os.path.exists('/content/koboldcpp/'):
    os.makedirs('/content/koboldcpp/')

# Checking if you already downloaded Kobold
if not os.path.exists("/content/koboldcpp.tar.gz"):
    if Force_Update_Build == False:
        response = requests.get(url, stream=True)
        filename = url.split("/")[-1]
        with open(filename, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)

        with tarfile.open(filename, 'r:gz') as tar:
            for member in tar.getmembers():
                if member.name.startswith('koboldcpp'):
                    try:
                        tar.extract(member, path='/content')
                    except Exception as e:
                        print(f"Error extracting '{member.name}': {str(e)}")

        print("File extraction to /content/ completed!")
    else:
        print("Skipping prebuilt kobold, will build manually...")
        !git clone https://github.com/LostRuins/koboldcpp
        %cd /content/koboldcpp
        !make LLAMA_CUBLAS=1

# Change to the directory
%cd /content/koboldcpp

# Hosting the cloudflared server
!wget -c https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!echo > nohup.out
!nohup ./cloudflared-linux-amd64 tunnel --url http://localhost:5001 &
!sleep 10
!cat nohup.out

# Download the file if it doesn't exist already
if not os.path.exists('/content/koboldcpp/model.gguf'):
    !wget $Model -O model.gguf

if os.path.exists('/content/koboldcpp/model.gguf'):
    !rm koboldcpp.py
    !wget https://cdn.discordapp.com/attachments/1132396046480314388/1150324006407376927/koboldcpp.py
    !python koboldcpp.py model.gguf --stream --usecublas 0 normal mmq --context $Context --ropeconfig 1.0 10000 --gpulayers $Layers --hordeconfig concedo
else:
    print("Failed to download the GGUF model. Please retry.")